# Excercise 1b: Extract acoustic features for Berlin Emodb

## Tasks:
0) fill in the XXX
1) add a markup cell above each code cell that comments what is happening in the code cell.
2) get a copy of the midlevel descriptors library (mld.zip from ISIS) and unzip in your seminar folder  

In [ ]:
import opensmile
import pandas as pd
import os
import sys
sys.path.append('./mld/src')
import midlevel_descriptors as mld

In [ ]:
# read in dataframe from previous exercise
df = pd.read_pickle(XXX)

In [ ]:
!mkdir store

In [ ]:
smile_feats_file = 'store/smile_feats.pkl'
if not os.path.isfile(smile_feats_file):
    smile_functionals = opensmile.Smile(
        feature_set = opensmile.FeatureSet.GeMAPSv01b,
        feature_level = opensmile.FeatureLevel.Functionals,
    )
    os_df = smile_functionals.process_files(df.index)
    os_df.index = os_df.index.droplevel(1)
    os_df.index = os_df.index.droplevel(1)
    os_df.to_pickle(smile_feats_file)
else:
    os_df = pd.read_pickle(smile_feats_file)

In [ ]:
fex_mld = mld.MLD()
mld_df = fex_mld.extract_from_index(index=df, cache_path='./mld_cache')

In [ ]:
df['mean_f0'] = os_df['F0semitoneFrom27.5Hz_sma3nz_amean']

In [ ]:
def st2hz(st):
    return 2 * st / 12 * 27.5
df['mean_f0_hz'] = df['mean_f0'].apply(st2hz)

In [ ]:
df['speech_rate'] = mld_df['hld_sylRate']

In [ ]:
df.to_pickle('store/emodb.pkl')

In [ ]:
# inspect the first three entries of the dataframe
df.head(XXX)

## Prepare a second dataframe with means (from sentence) per speaker and emotion.

In [ ]:
# remember the sexes
sexes = {}
for spk in df.speaker.unique():
    sex = df.loc[df['speaker'] == spk, 'sex'].unique()[0]
    sexes.update({spk: sex})
print(sexes)

In [ ]:
# group by speaker and emotion, neglecting sex and transcription
df_mean = df.groupby(['speaker', 'emotion']).mean()
df_mean = df_mean.reset_index()
# add the sex
df_mean['sex'] = df_mean['speaker'].apply(lambda x: sexes.get(x))


In [ ]:
# reorder
cols = list(df_mean)
cols.insert(2, cols.pop(cols.index('sex')))
df_mean = df_mean.loc[:, cols]

In [ ]:
# inspect the shapes
print(df.shape, df_mean.shape)

In [ ]:
df_mean.head(1)

In [ ]:
df_mean.to_pickle('store/emodb_means.pkl')

## Make a third dataframe where the missing values are replaced by mean values (per emotion and speaker)

In [ ]:
speakers = df.speaker.unique()
transcriptions = df.transcription.unique()
emotions = df.emotion.unique()

In [ ]:
from itertools import product
df_all = pd.DataFrame(list(product(speakers, emotions, transcriptions)), columns=['speaker', 'emotion', 'transcription'])
df_all['mean_f0'] = None
df_all['mean_f0_hz'] = None
df_all['speech_rate'] = None

In [ ]:
def get_val (feat, emotion, speaker, transcript):
    """get the first value for speaker, emotion and transcript"""
    c1 = df['emotion']==emotion
    c2 = df['speaker'] == speaker
    c3 = df['transcription'] == transcript 
    try:
        return df.loc[c1 & c2 & c3, feat][0]
    except IndexError:
        # if the value is not there, use the mean val from the other transcripts
        mean  = df.loc[c1 & c2, feat].mean()
        if pd.isna(mean):
            # if there aren't any values for this speaker and emotion, use the mean for his/her sex and emotion
            ssex = sexes.get(speaker)
            c4 = df.sex==ssex
            mean = df.loc[c1 & c4, feat].mean()    
        return mean


In [ ]:
features = ['mean_f0', 'mean_f0_hz', 'speech_rate']

for speaker in speakers:
    for emotion in emotions:
        for transcript in transcriptions:
            for feat in features:
                c1 = df_all['emotion']==emotion
                c2 = df_all['speaker'] == speaker
                c3 = df_all['transcription'] == transcript
                df_all.loc[c1 & c2 & c3, feat] = get_val(feat, emotion, speaker, transcript)

In [ ]:
# add the sex
df_all['sex'] = df_all['speaker'].apply(lambda x: sexes.get(x))
# reorder
cols = list(df_all)
cols.insert(2, cols.pop(cols.index('sex')))
df_all = df_all.loc[:, cols]

In [ ]:
# make it numbers
df_all.mean_f0 = pd.to_numeric(df_all.mean_f0)
df_all.speech_rate = pd.to_numeric(df_all.speech_rate)
df_all.mean_f0_hz = pd.to_numeric(df_all.mean_f0_hz)

In [ ]:
df_all.to_pickle('store/emodb_all.pkl')

In [ ]:
# print the shapes of all three dataframes
print (XXX)

In [ ]:
df_all.isna().any()